In [1]:
import numpy as np 
import pandas as pd 

countries_report = pd.read_csv("countries.csv", header = 0, low_memory=False)
happiness_report = pd.read_csv("2017.csv", header = 0, low_memory=False)

happiness_report = happiness_report.sample(frac=1)
print happiness_report.shape
print countries_report.shape

(151, 12)
(227, 20)


In [2]:
happy_data = []
country_data = []

rows = happiness_report.shape[0]
for i in range(rows):
    key = unicode("Country", "utf-8")
    c = happiness_report.loc[happiness_report.index[[i]], key]
    c_data = happiness_report.loc[happiness_report.index[[i]]]
#     print c_data.values[0]
    happy_data.append(c_data.values[0])
    
    # Gets corresponding countries.csv data for each country in 2017.ccsv
    Country = happiness_report.loc[happiness_report.index[[i]], key].values
#     print Country[0]
    
    col = countries_report['Country']
    index = -1
    for i in col:
        index += 1
        i = str(i)
        if Country[0] in i:
#             print i
            idx = index
            break
        
    data = countries_report.loc[countries_report.index[idx]]
    country_data.append(data)

In [3]:
print len(happy_data)
print len(country_data)

#sanity check
idx = 0
for i in country_data:
    print happy_data[idx][0],
    print i[0]
    idx += 1

151
151
Nepal Nepal 
Tajikistan Tajikistan 
Dominican Republic Dominican Republic 
Kenya Kenya 
Azerbaijan Azerbaijan 
Malaysia Malaysia 
Uganda Uganda 
Taiwan Taiwan 
Yemen Yemen 
Poland Poland 
Tunisia Tunisia 
Armenia Armenia 
Uruguay Uruguay 
Slovakia Slovakia 
Albania Albania 
Jordan Jordan 
Ethiopia Ethiopia 
Japan Japan 
Bulgaria Bulgaria 
Mexico Mexico 
Qatar Qatar 
Portugal Portugal 
Indonesia Indonesia 
United Kingdom United Kingdom 
Sri Lanka Sri Lanka 
South Africa South Africa 
Morocco Morocco 
Lithuania Lithuania 
Thailand Thailand 
Libya Libya 
Tanzania Tanzania 
Turkey Turkey 
Uzbekistan Uzbekistan 
Bhutan Bhutan 
Liberia Liberia 
Norway Norway 
Benin Benin 
Mauritania Mauritania 
France France 
Somalia Somalia 
Hungary Hungary 
Paraguay Paraguay 
Belize Belize 
Netherlands Netherlands 
Italy Italy 
Burundi Burundi 
Peru Peru 
Sudan Sudan 
Bosnia and Herzegovina Bosnia and Herzegovina 
Argentina Argentina 
Honduras Honduras 
Zimbabwe Zimbabwe 
Greece Greece 
Denmark Den

In [13]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

# print happy_data
happyX = []
happyY = []

for i in happy_data:
    happyX.append(i[0])
    happyY.append(i[2])
    
happyX = np.array(happyX)
happyY = np.array(happyY)
print happyX.shape
print happyY.shape

def mse_baseline(y_train, y_test):
    avg = np.average(y_train)
    y_predict = np.full(y_test.shape, avg)
    print "baseline", mean_squared_error(y_test, y_predict)
    return mean_squared_error(y_test, y_predict)


errors = []
kf = KFold(n_splits=5)
for train_index, test_index in kf.split(happyX):
#     print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = happyX[train_index], happyX[test_index]
    y_train, y_test = happyY[train_index], happyY[test_index]
    errors.append(mse_baseline(y_train, y_test))
print sum(errors) / len(errors)

(151,)
(151,)
baseline 0.773808921175
baseline 1.39812153699
baseline 1.30299367031
baseline 1.55261571674
baseline 1.41443669044
1.28839530713


In [5]:
import math
allArr = np.zeros(19)
for i in country_data:
    for j in range(2,19):
        if isinstance(i[j], float) or isinstance(i[j], int):
            if not math.isnan(i[j]):
                allArr[j] += i[j]
        else:
            allArr[j] +=(float(i[j].replace(',','.')))
means = allArr/float(len(country_data))

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn import preprocessing

countriesX = [] 

# train using one column from 
# for i in country_data:
#     #comment this line out if not float data
#     f = float(i[7].replace(',','.'))
#     countriesX.append([f])
    
for i in country_data:
    info = []
    for j in [7,8,10]: 
        if isinstance(i[j], float) or isinstance(i[j], int):
            if not math.isnan(i[j]):
                info.append(i[j])
            else:
                info.append(means[j])
        else:
            info.append(float(i[j].replace(',','.')))  
    countriesX.append(info)
    
    
countriesX = np.array(countriesX)
print countriesX.shape
print happyY.shape
# lab_enc = preprocessing.LabelEncoder()
# happyY = lab_enc.fit_transform(happyY)
    
def mse_clf(X_train, X_test, y_train, y_test):
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    y_predict = lr.predict(X_test)
    print "Linear Regression", mean_squared_error(y_test, y_predict)
    return mean_squared_error(y_test, y_predict)
    
errors = []
kf = KFold(n_splits=5)
for train_index, test_index in kf.split(countriesX):
#     print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = countriesX[train_index], countriesX[test_index]
    y_train, y_test = happyY[train_index], happyY[test_index]
    errors.append(mse_clf(X_train, X_test, y_train, y_test))
print sum(errors) / len(errors)

(151, 3)
(151,)
Linear Regression 0.378997494063
Linear Regression 0.528980012956
Linear Regression 0.570917410822
Linear Regression 0.5396928925
Linear Regression 0.338297461796
0.471377054427
